In [1]:
from openai import OpenAI
import pandas as pd
import os
import json
from dotenv import load_dotenv

# Load the API key from your .env
load_dotenv(dotenv_path=".env")  # You can give it the path explicitly
# Define api_key first
api_key = os.getenv("OPENAI_API_KEY")
org_id = "org-k2Sv4lKBPfhKSPP44JoM0f3X" 

# Then initialize the OpenAI client
client = OpenAI(api_key=api_key)


In [2]:
print("Loaded API key:", os.getenv("OPENAI_API_KEY")[:8] + "..." if os.getenv("OPENAI_API_KEY") else "❌ Not loaded")

Loaded API key: sk-proj-...


In [3]:

# Load the CSV file
df = pd.read_csv('Data/maintenance.csv')

# Display the first few rows of the DataFrame
df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Vibration Levels,Operational Hours,Failure Type
0,1,M14860,M,298.1,308.6,1551,42.8,42.0,20.0,No Failure
1,2,L47181,L,298.2,308.7,1408,46.3,52.0,21.0,No Failure
2,3,L47182,L,298.1,308.5,1498,49.4,44.0,18.0,No Failure
3,4,L47183,L,298.2,308.6,1433,39.5,52.0,10.0,No Failure
4,5,L47184,L,298.2,308.7,1408,40.0,44.0,10.0,No Failure


In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 500 entries, 0 to 499
Data columns (total 10 columns):
 #   Column                   Non-Null Count  Dtype  
---  ------                   --------------  -----  
 0   UDI                      500 non-null    int64  
 1   Product ID               500 non-null    object 
 2   Type                     500 non-null    object 
 3   Air temperature [K]      500 non-null    float64
 4   Process temperature [K]  500 non-null    float64
 5   Rotational speed [rpm]   500 non-null    int64  
 6   Torque [Nm]              500 non-null    float64
 7   Vibration Levels         500 non-null    float64
 8   Operational Hours        500 non-null    float64
 9   Failure Type             500 non-null    object 
dtypes: float64(5), int64(2), object(3)
memory usage: 39.2+ KB


In [5]:
df.isnull().sum()

UDI                        0
Product ID                 0
Type                       0
Air temperature [K]        0
Process temperature [K]    0
Rotational speed [rpm]     0
Torque [Nm]                0
Vibration Levels           0
Operational Hours          0
Failure Type               0
dtype: int64

In [6]:
df.duplicated().sum()

np.int64(0)

In [7]:
df.dtypes

UDI                          int64
Product ID                  object
Type                        object
Air temperature [K]        float64
Process temperature [K]    float64
Rotational speed [rpm]       int64
Torque [Nm]                float64
Vibration Levels           float64
Operational Hours          float64
Failure Type                object
dtype: object

In [8]:
# Removes any leading/trailing whitespace from column names to ensure consistent access
df.columns = df.columns.str.strip()


In [9]:
print(df.columns.tolist())


['UDI', 'Product ID', 'Type', 'Air temperature [K]', 'Process temperature [K]', 'Rotational speed [rpm]', 'Torque [Nm]', 'Vibration Levels', 'Operational Hours', 'Failure Type']


In [10]:
# Generates a formal technician comment string for a given engine row.
# Summarizes key metrics like RPM, torque, temperature, vibration, and operational hours,
# and appends a maintenance recommendation based on the failure type.
def generate_inspection_comment(row):
    engine = row['UDI']
    air_temp = row['Air temperature [K]']
    proc_temp = row['Process temperature [K]']
    rpm = row['Rotational speed [rpm]']
    torque = row['Torque [Nm]']
    vibration = row['Vibration Levels']
    hours = row['Operational Hours']
    failure = row['Failure Type']

    comment = (
        f"Technician inspected Engine #{engine}. RPM at {rpm}, torque at {torque:.1f} Nm. "
        f"Air temp {air_temp:.1f}K, Process temp {proc_temp:.1f}K. "
        f"Vibration level recorded at {vibration:.1f}. Engine has run for {hours:.1f} hours. "
    )

    if failure == "No Failure":
        comment += "No failure detected."
    elif "Tool" in failure:
        comment += "Signs of tool wear observed. Recommend inspection."
    else:
        comment += "Anomaly present. Further diagnostics required."

    return comment



In [11]:
# If your original cleaned DataFrame is named clean_df:
comment_df = df.copy()

comment_df["Comment"] = comment_df.apply(generate_inspection_comment, axis=1)



In [12]:

comment_df.to_csv('Data/maintenance_comments.csv', index=False)


In [13]:
comment_df.head()

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Vibration Levels,Operational Hours,Failure Type,Comment
0,1,M14860,M,298.1,308.6,1551,42.8,42.0,20.0,No Failure,"Technician inspected Engine #1. RPM at 1551, t..."
1,2,L47181,L,298.2,308.7,1408,46.3,52.0,21.0,No Failure,"Technician inspected Engine #2. RPM at 1408, t..."
2,3,L47182,L,298.1,308.5,1498,49.4,44.0,18.0,No Failure,"Technician inspected Engine #3. RPM at 1498, t..."
3,4,L47183,L,298.2,308.6,1433,39.5,52.0,10.0,No Failure,"Technician inspected Engine #4. RPM at 1433, t..."
4,5,L47184,L,298.2,308.7,1408,40.0,44.0,10.0,No Failure,"Technician inspected Engine #5. RPM at 1408, t..."


In [14]:
# Prompt builder: structured JSON output
def create_prompt(row):
    return (
        f"You are a maintenance technician writing a short inspection report. "
        f"Based on the engine data below, respond in JSON format with two keys: "
        f"'comment' (a short summary in technician tone) and 'action' (a clear recommended action).\n\n"
        f"- Engine ID: {row['UDI']}\n"
        f"- Air temperature: {row['Air temperature [K]']} K\n"
        f"- Process temperature: {row['Process temperature [K]']} K\n"
        f"- Rotational speed: {row['Rotational speed [rpm]']} rpm\n"
        f"- Torque: {row['Torque [Nm]']} Nm\n"
        f"- Vibration level: {row['Vibration Levels']}\n"
        f"- Operational hours: {row['Operational Hours']}\n"
        f"- Failure Type: {row['Failure Type']}\n\n"
        f"Respond in this exact JSON format:\n"
        f'{{"comment": "...", "action": "..."}}'
    )


In [15]:
# 🤖 LLM call + JSON parser (OpenAI v1)
def get_llm_comment(prompt):
    response = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=200
    )
    raw_output = response.choices[0].message.content.strip()

    try:
        result = json.loads(raw_output)
        return result.get("comment", ""), result.get("action", "")
    except json.JSONDecodeError:
        return raw_output, "Unable to extract action"


In [16]:
#  Make a safe copy of the original dataframe
llm_comment_df = df.copy()


In [17]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[{"role": "user", "content": "Say hi like a technician"}],
    temperature=0.3,
    max_tokens=50
)

print(response.choices[0].message.content.strip())


Hello! How can I assist you today? If you're experiencing any issues or need support, I'm here to help troubleshoot and resolve any technical concerns you might have.


In [18]:
def safe_llm_comment(row):
    try:
        return pd.Series(get_llm_comment(create_prompt(row)))
    except Exception as e:
        print(f"Error on row {row['UDI']}: {e}")
        return pd.Series(["LLM error", "Unable to generate action"])


In [ ]:
# Apply the LLM prompt and response function to each row of the dataset.
# For every row, it generates a structured JSON response from the model,
# then extracts and stores the 'comment' and 'recommended action' into two new columns.
llm_comment_df[["LLM Comment", "Recommended Action"]] = llm_comment_df.apply(
    lambda row: pd.Series(get_llm_comment(create_prompt(row))),
    axis=1
)


In [32]:

llm_comment_df.head()


,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Vibration Levels,Operational Hours,Failure Type,LLM Comment,Recommended Action,Action Category
0,1,M14860,M,298.1,308.6,1551,42.8,42.0,20.0,No Failure,Engine ID 1 is operating within normal paramet...,Continue regular monitoring and schedule the n...,Monitor
1,2,L47181,L,298.2,308.7,1408,46.3,52.0,21.0,No Failure,Engine ID 2 is operating within normal paramet...,Monitor vibration levels and perform routine m...,Monitor
2,3,L47182,L,298.1,308.5,1498,49.4,44.0,18.0,No Failure,Engine ID 3 is operating within normal paramet...,Continue regular monitoring and maintenance sc...,Monitor
3,4,L47183,L,298.2,308.6,1433,39.5,52.0,10.0,No Failure,Engine ID 4 is operating within normal paramet...,Monitor vibration levels during next inspectio...,Monitor
4,5,L47184,L,298.2,308.7,1408,40.0,44.0,10.0,No Failure,Engine ID 5 is operating within normal paramet...,Monitor vibration levels and conduct a follow-...,Monitor


In [33]:
llm_comment_df.to_csv("Data/llm_comments_with_actions.csv", index=False)

In [24]:
#check the CSV
llm_comment_df.sample(5)[["UDI", "LLM Comment", "Recommended Action"]]


,UDI,LLM Comment,Recommended Action
304,305,Engine 305 is operating within normal paramete...,Monitor vibration levels during next inspectio...
318,319,Engine ID 319 is operating within normal param...,Continue regular monitoring and schedule the n...
254,255,Engine ID 255 shows signs of overstrain failur...,Conduct a thorough inspection of the engine co...
101,102,Engine ID 102 is operating within normal param...,Monitor vibration levels and schedule a follow...
453,454,Engine ID 454 shows signs of tool wear failure...,Schedule immediate inspection and maintenance ...


In [39]:
def bucket_action(text):
    text = text.lower()
    if "no action" in text or "stable" in text:
        return "No Action"
    elif "monitor" in text:
        return "Monitor"
    elif any(term in text for term in ["inspection", "follow-up", "review", "diagnostics", "replace"]):
        return "Flag for Inspection"
    else:
        return "Other"


llm_comment_df["Action Category"] = llm_comment_df["Recommended Action"].apply(bucket_action)
llm_comment_df["Action Category"].value_counts()


Action Category
Monitor                468
Flag for Inspection     30
Other                    2
Name: count, dtype: int64

In [40]:
action_df= llm_comment_df.copy() 

In [44]:
action_df.sample(10) 

,UDI,Product ID,Type,Air temperature [K],Process temperature [K],Rotational speed [rpm],Torque [Nm],Vibration Levels,Operational Hours,Failure Type,LLM Comment,Recommended Action,Action Category
306,307,L47486,L,297.8,308.4,1374,47.1,34.0,96.90,No Failure,Engine ID 307 is operating within normal param...,Continue regular monitoring and maintenance sc...,Monitor
313,314,L47493,L,298.0,308.6,1313,50.8,33.0,99.38,No Failure,Engine ID 314 is operating within normal param...,Continue regular monitoring and maintenance sc...,Monitor
181,182,H29595,H,298.2,308.3,1824,24.2,34.0,52.58,No Failure,Engine ID 182 is operating within normal param...,Continue regular monitoring and maintenance sc...,Monitor
29,30,L47209,L,299.0,309.4,1693,30.1,39.0,25.38,No Failure,Engine 30 is operating within normal parameter...,Continue regular monitoring and maintenance as...,Monitor
23,24,L47203,L,299.0,309.4,1758,25.7,36.0,23.59,No Failure,Engine ID 24 is operating within normal parame...,Continue regular monitoring and maintenance sc...,Monitor
477,478,H29891,H,297.4,309.0,1468,41.4,24.0,157.53,No Failure,Engine 478 is operating within normal paramete...,Monitor vibration levels during next inspectio...,Monitor
344,345,M15204,M,297.5,308.2,2448,13.8,33.0,110.37,No Failure,Engine ID 345 is operating within normal param...,Continue regular monitoring and schedule next ...,Monitor
346,347,M15206,M,297.5,308.3,1459,45.2,24.0,111.08,No Failure,Engine ID 347 is operating within normal param...,Monitor vibration levels during next inspectio...,Monitor
388,389,L47568,L,297.6,308.7,1455,48.2,44.0,125.97,No Failure,Engine ID 389 is operating within normal param...,Continue regular monitoring and schedule the n...,Monitor
481,482,M15341,M,297.3,309.0,1556,39.5,36.0,158.95,No Failure,Engine ID 482 is operating within normal param...,Continue regular monitoring and schedule next ...,Monitor


In [45]:
action_df.to_csv("Data/llm_comments_with_actions.csv", index=False)

In [ ]:
action_df[action_df["Flag for Review"]].groupby("Product ID").size().sort_values(ascending=False)


KeyError: 'Flag for Review'

In [ ]:
def get_llm_comment(prompt):
    response = openai.ChatCompletion.create(
        model="gpt-4o-mini",  
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3,
        max_tokens=150
    )
    return response['choices'][0]['message']['content'].strip()


In [ ]:
def generate_sap_output(comment):
    prompt = f"""
You are a smart SAP Plant Maintenance assistant. Read this technician inspection comment and convert it into a structured SAP breakdown notification.

Comment: "{comment}"

Return the following fields:
Asset:
Issue:
Priority:
Action:
Notification Type:
Status:
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response['choices'][0]['message']['content']

In [33]:
for i in range(3):
    print(generate_sap_output(df['Comment'].iloc[i]))

APIRemovedInV1: 

You tried to access openai.ChatCompletion, but this is no longer supported in openai>=1.0.0 - see the README at https://github.com/openai/openai-python for the API.

You can run `openai migrate` to automatically upgrade your codebase to use the 1.0.0 interface. 

Alternatively, you can pin your installation to the old version, e.g. `pip install openai==0.28`

A detailed migration guide is available here: https://github.com/openai/openai-python/discussions/742


In [ ]:

# ✅ Your OpenAI API key here
openai.api_key = "sk-xxxxxxx"

# 🔧 The GenAI function
def generate_work_order(comment):
    prompt = f"""You are a maintenance assistant. Convert inspection comments into structured SAP-style work orders.

Examples:

Comment: "Pump 7 leaking from main valve"
→
Asset: Pump 7
Issue: Leak at main valve
Priority: High
Action: Replace valve gasket
Notification Type: M2 (Breakdown)
Status: Open

Comment: "Conveyor belt jammed overnight"
→
Asset: Conveyor
Issue: Jam
Priority: Medium
Action: Inspect rollers, remove obstruction
Notification Type: M2 (Breakdown)
Status: Open

Now process this:
Comment: "{comment}"
→
"""
    response = openai.ChatCompletion.create(
        model="gpt-4",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    return response['choices'][0]['message']['content']

# 🖥️ Gradio demo
demo = gr.Interface(
    fn=generate_work_order,
    inputs="text",
    outputs="text",
    title="GenAI SAP Work Order Generator",
    description="Paste any maintenance comment or inspection log to convert it into a structured work order."
)

demo.launch()